In [48]:
# Importing Dependencies
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

# **Data Cleaning**
### **Data Ingestion**


In [49]:
# Load the data
job_postings = pd.read_csv('../../../data/job_postings.csv')
job_skills = pd.read_csv('../../../data/job_skills.csv')

## **Title Classification (input for ticket 1.1)**
### **Target Jobs Classification Regex**
#### Input job_postings.csv dataframe with job_title column

In [50]:
# target job title regex list
target_job_titles_regex = {
    "MLOps Engineer": r"(?i)(MLOps|Machine Learning Operations|Machine Learning Infrastructure Engineer|ML Infrastructure|ML Platform|ML Systems|ML Platform Engineer|AIML Ops Engineer|Machine Learning Software Developer)\w*[-\s]?",

    "Machine Learning Engineer": r"(?i)(Machine Learning Engineer|ML Engineer|Machine Learning Engineering|ML Developer|Machine Learning Software Engineer|AIML Engineer|AIML Data Scientist|AI Data Science Lead)\w*[-\s]?",

    "Data Architect": r"(?i)(Data Architect|Senior Data Architect|Cloud Data Architect|Big Data Architect|Enterprise Data Architect|Principal Data Architect|Lead Data Architect|Data Warehouse Architect|Data Architecture|Data Lake Architect|Data Streaming Architect)\w*[-\s]?",

    "Database Engineer / Administrator": r"(?i)(Database|Database Architect|DBA\b|Cloud Database|Azure Database|AWS Database|Databases|GCP Database|PostgreSQL|NoSQL|Oracle Database Engineer)\w*[-\s]?",

    "Data Engineer": r"(?i)(Data Engineer|Senior Data Engineer|Lead Data Engineer|Big Data Engineer|Data Engineering|Data Engineering Manager|Data Engineering Architect|Data Pipeline Engineer|Big Data Developer|Data Engineers|Data Integrations|Data Infrastructure|ETL Developer)\w*[-\s]?",

    "Data Governance & Security": r"(?i)(Data Governance|Data Privacy|Data Steward|Data Protection|Data Security|Master Data Management|Data Governance Manager|Data Compliance|Data Lifecycle Manager)\w*[-\s]?",

    "Data Operations & Management": r"(?i)(Data Manager|Enterprise Data Manager|Data Operations|Data Operations Manager|Data Operations Analyst|Data Management Engineer|Data Strategy Manager|Data Solution Architect|Data Deployment|Data Conversion|Data Replication Engineer|DevOps Engineer|Distributed Systems|Storage)\w*[-\s]?",

    "Data Modeling & Warehousing": r"(?i)(Data Modeling|Data Warehouse|Big Data Developer|Data Warehouse Architect|Cloud Datawarehouse|Data Platform Developer)\w*[-\s]?",

    "Data Specialist": r"(?i)(Data Specialist|Data Processing|Data Consultant|Data Quality Manager|Data Coordinator|Data Entry Specialist)\w*[-\s]?",

    "Data Scientist": r"(?i)(Data Scientist|Data Scientists|Data Science Engineer|Data Science Manager|Data Science Analyst|Data Science Practitioner|Customer Data Scientist)\w*[-\s]?",

    "Data Analyst": r"(?i)(Data Analyst|Data Analysts|Financial Data Analyst|Business Intelligence|BI Analyst|Data Business Analyst|Data Insights Analyst)\w*[-\s]?",

    "Software & Platform Engineering": r"(?i)(Software Engineer|Software Engineering|Software Developer|Software Engineer Data Science|Software Engineer Data Platforms|Platform Engineer|Application Developer|Backend Engineer|Systems Developer)\w*[-\s]?",

    "Cloud & Infrastructure Engineering": r"(?i)(Cloud Data|Cloud Data Architect|Azure Data|AWS Data|Azure Databricks|AWS Databricks|Cloud Engineer|Cloud Platform Engineer|Infrastructure Engineer|Datacenter Technician|Datacenter Engineer|Datacenter Network Engineer|Datacenter Engineering|Site Reliability Engineer|SRE)\w*[-\s]?",

    "Risk & Compliance Analytics": r"(?i)(Risk Analyst|AML\b|BSA|Risk Modeling|Financial Analyst|Hedge Fund|Data Loss Prevention|DLP)\w*[-\s]?"
}


### **Classification Function**

In [51]:
# Function to Classify Job Titles
def classify(job_title, keywords_list=target_job_titles_regex):
    for industry, keyword in keywords_list.items():
        match = re.search(keyword, str(job_title))
        if match:
            keyword = re.sub(r'[^a-zA-Z\s]', '', match.group()).strip().title()   # using match.group() to return the actual keyword that was matched rather than the regex pattern
            return industry, keyword              
    return "unclassified", "unclassified"


In [52]:
# Copy Dataframe and Execute the classification function
classified_job_titles = job_postings.copy()
classified_job_titles['job_classification'], classified_job_titles['job_keyword'] = zip(*classified_job_titles['job_title'].apply(classify))

# Check the results
# classified_job_titles.head(10)


### **Seniority Level Classification Regex**

In [53]:
# seniority level regex list
seniority_levels_regex = {
    # 🔹 Principal / Staff-Level Roles (Must Be Checked First)
    "Principal / Staff-Level": r"(?i)(Principal|Staff|Sr[-\s]?Staff|Distinguished|Fellow|Master|L4|Level 4|Chief[-\s]?Architect|Chief[-\s]?Scientist)\w*[-\s]?",

    # 🔹 Lead / Supervisor Roles (Checked Before Senior)
    "Lead": r"(?i)(Lead|Tech[-\s]?Lead|Team[-\s]?Lead|Supervisor|Group[-\s]?Lead|Project[-\s]?Lead|Engineering[-\s]?Lead|Squad[-\s]?Lead|Chapter[-\s]?Lead|Manager|Head[-\s]?of[-\s]?Team)\w*[-\s]?",

    # 🔹 Senior-Level Roles (Checked Before Mid-Level)
    "Senior-Level": r"(?i)(Senior|Sr\.?|SNR|SEN|L3|Level 3|Expert|Specialist|Advanced|Seasoned|Experienced)\w*[-\s]?",

    # 🔹 Mid-Level Roles (Checked Before Junior)
    "Mid-Level": r"(?i)(Mid[-\s]?Level|Intermediate|Mid|L2|Level 2|Professional|Regular)\w*[-\s]?",

    # 🔹 Entry-Level / Junior Roles (Checked After Principal & Senior)
    "Entry-Level / Junior": r"(?i)(Junior|Jr\.?|Entry[-\s]?Level|Associate|Graduate|Trainee|Fresher|New Grad|Early[-\s]?Career|L1|Level 1)\w*[-\s]?",

    # 🔹 Intern / Internship Roles (Checked Last)
    "Intern": r"(?i)(Intern|Internship|Co[-\s]?Op|Apprentice|Trainee)\w*[-\s]?",

    # 🔹 Director / Executive Roles (Checked Last for Highest Priority)
    "Director / Executive": r"(?i)(Director|Head|VP|Vice[-\s]?President|CIO|CTO|CISO|CEO|Chief|Executive|C[-]?Level|Managing[-\s]?Director|Global[-\s]?Head|President|Founder|Partner)\w*[-\s]?"
}


### **Seniority Level Classification Function**

In [54]:
# Create single use function to classify seniority level
def classify_seniority_level(job_title):
    return classify(job_title, seniority_levels_regex)

In [55]:
# Execute the classification function on the copied dataframe
classified_job_titles['seniority_level'], classified_job_titles['seniority_level_keyword'] = zip(*classified_job_titles['job_title'].apply(classify_seniority_level))

# Check the results
# classified_job_titles.head()

### **Check Group**

In [56]:
# check group
job_check = 'Cloud & Infrastructure Engineering'
# pd.DataFrame(classified_job_titles[classified_job_titles['job_classification'] == job_check].head())

### **Drop unclassified rows**
#### Title Classification Section - Outputs Classifieds Dataframe with Added Job Classification & Seniority Level Columns


In [57]:
# Dropping rows
classified_job_titles = classified_job_titles[classified_job_titles['job_classification'] != 'unclassified']
classified_job_titles = classified_job_titles[classified_job_titles['seniority_level'] != 'unclassified']

# check results
classified_job_titles

,job_link,last_processed_time,last_status,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type,job_classification,job_keyword,seniority_level,seniority_level_keyword
0,https://www.linkedin.com/jobs/view/senior-mach...,2024-01-21 08:08:48.031964+00,Finished NER,t,t,f,Senior Machine Learning Engineer,Jobs for Humanity,"New Haven, CT",2024-01-14,East Haven,United States,Agricultural-Research Engineer,Mid senior,Onsite,Machine Learning Engineer,Machine Learning Engineer,Senior-Level,Senior
1,https://www.linkedin.com/jobs/view/principal-s...,2024-01-20 04:02:12.331406+00,Finished NER,t,t,f,"Principal Software Engineer, ML Accelerators",Aurora,"San Francisco, CA",2024-01-14,El Cerrito,United States,Set-Key Driver,Mid senior,Onsite,Software & Platform Engineering,Software Engineer,Principal / Staff-Level,Principal
2,https://www.linkedin.com/jobs/view/senior-etl-...,2024-01-21 08:08:31.941595+00,Finished NER,t,t,f,Senior ETL Data Warehouse Specialist,Adame Services LLC,"New York, NY",2024-01-14,Middletown,United States,Technical Support Specialist,Associate,Onsite,Data Modeling & Warehousing,Data Warehouse,Senior-Level,Senior
3,https://www.linkedin.com/jobs/view/senior-data...,2024-01-20 15:30:55.796572+00,Finished NER,t,t,f,Senior Data Warehouse Developer / Architect,Morph Enterprise,"Harrisburg, PA",2024-01-12,Lebanon,United States,Architect,Mid senior,Onsite,Data Modeling & Warehousing,Data Warehouse,Senior-Level,Senior
4,https://www.linkedin.com/jobs/view/lead-data-e...,2024-01-21 08:08:58.312124+00,Finished NER,t,t,f,Lead Data Engineer,Dice,"Plano, TX",2024-01-14,McKinney,United States,Maintenance Data Analyst,Mid senior,Onsite,Data Engineer,Lead Data Engineer,Lead,Lead
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12207,https://uk.linkedin.com/jobs/view/senior-lead-...,2024-01-21 07:29:06.068335+00,Finished NER,t,t,f,Senior/Lead Data Software Engineer,EPAM Systems,"London, England, United Kingdom",2024-01-16,Slough,United Kingdom,Loading-Machine Tool-Setter,Associate,Onsite,Software & Platform Engineering,Software Engineer,Lead,Lead
12211,https://ca.linkedin.com/jobs/view/senior-data-...,2024-01-20 05:10:03.58781+00,Finished NER,t,t,f,Senior Data Insights Analyst,CARFAX Canada,"London, Ontario, Canada",2024-01-14,London,Canada,Data Entry Clerk,Mid senior,Onsite,Data Analyst,Data Insights Analyst,Senior-Level,Senior
12213,https://www.linkedin.com/jobs/view/corporate-a...,2024-01-19 15:10:41.177008+00,Finished NER,t,t,f,Corporate AML Alert Investigation Specialist,"Glacier Bancorp, Inc.","Kalispell, MT",2024-01-14,Montana,United States,Teller,Mid senior,Onsite,Risk & Compliance Analytics,Aml,Senior-Level,Specialist
12214,https://www.linkedin.com/jobs/view/senior-data...,2024-01-20 15:20:19.036168+00,Finished NER,t,t,f,Senior Data Scientist,Highnote,"San Francisco, CA",2024-01-16,San Rafael,United States,Mathematician,Mid senior,Onsite,Data Scientist,Data Scientist,Senior-Level,Senior
